In [1]:
%load_ext autoreload
%autoreload 2

from langchain.prompts import Prompt, BaseChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from tqdm import tqdm
import re
import pandas as pd
import dotenv
import random
import networkx as nx
from langchain.schema import (
    BaseMessage, 
    HumanMessage, 
    SystemMessage,
    AIMessage
)
dotenv.load_dotenv()

True

In [74]:
pairs_df = pd.read_csv('data/acm_ccs_clean.csv')

In [75]:
def get_nodes_edges(pairs_df, group):
    pairs = pairs_df[pairs_df['group'] == group]
    edges = []
    nodes = set()
    for i, row in pairs.iterrows():
        parent, child = row['parent'], row['child']
        parent = parent.replace('_', ' ')
        child = child.replace('_', ' ')
        edges.append({
            'parent': parent,
            'child': child,
        })
        nodes.add(parent)
        nodes.add(child)

    nodes = list(nodes)    
    # randomly shffle the nodes and relations to avoid learning pattern
    random.shuffle(edges)
    random.shuffle(nodes)

    return nodes, edges

def get_groups(pairs_df, split='train'):
    groups = pairs_df[pairs_df['type'] == split]['group'].unique()
    return groups

In [76]:


class TaxomomyPrompt(BaseChatPromptTemplate):
    def format_messages(self, **kwargs) -> list[BaseMessage]:
        group_examples = kwargs['group_examples']
        concepts = kwargs['concepts']

        prefix_prompt = (
            "You are an expert constructing a taxonomy from a list of concepts. Given a list of concepts,"
            "construct a taxonomy by creating a list of their parent-child relationships.\n\n"
        )

        prefix_message = SystemMessage(content=prefix_prompt)

        example_messages = []
        for nodes, edges in group_examples:
            node_prompt = '; '.join(nodes)
            edge_prompt = '; '.join([f"{edge['child']} is a subtopic of {edge['parent']}" for edge in edges])

            example_messages.append(
                HumanMessage(content=f"Concepts: {node_prompt}\nRelationships: ")
            )

            example_messages.append(
                AIMessage(content=f"{edge_prompt}\n\n")
            )
        
        concepts = '; '.join(concepts)
        question_message = HumanMessage(content=f"Concepts: {concepts}\nRelationships: ")


        return [prefix_message] + example_messages + [question_message]


In [78]:
few_shot_numbers = 5

train_groups = get_groups(pairs_df)
test_groups = get_groups(pairs_df, split='test')

def generate_train_examples(pairs_df, num_examples=5):
    random.shuffle(train_groups)
    train_group_examples = []
    for group in train_groups[:num_examples]:
        nodes, edges = get_nodes_edges(pairs_df, group)
        train_group_examples.append((nodes, edges))
    return train_group_examples
    

In [85]:
class TaxonomyParser(BaseOutputParser):
    def parse(self, output):
        output = output.split(';')
        output = [line for line in output if line != '']
        output = [re.split(' is a subtopic of ', line) for line in output]
        print(output)
        result = []
        for line in output:
            if len(line) != 2:
                continue
            result.append({
                'child': line[0].strip(),
                'parent': line[1].strip()
            })
        return result
    
def call_chain(chain, concepts, group_examples, num_retries=5):
    count = 0
    while count < num_retries:
        try:
            result = chain.run(
                concepts=concepts,
                group_examples=group_examples,
            )
            return result
        except Exception as e:
            print(e)
            count += 1
    raise Exception("Failed to generate result")

prompt = TaxomomyPrompt(
    input_variables=['concepts', 'group_examples'],
)
llm = ChatOpenAI(model='gpt-4')# ChatOpenAI(model='gpt-3.5-turbo')
chain = LLMChain(llm=llm, prompt=prompt, output_parser=TaxonomyParser())
results = []

In [86]:
for test_group in tqdm(test_groups):
    concepts, edges = get_nodes_edges(pairs_df, test_group)
    group_examples = generate_train_examples(pairs_df, few_shot_numbers)

    print(prompt.format(
        concepts=concepts,
        group_examples=group_examples,
    ))
    break

  0%|          | 0/15 [00:00<?, ?it/s]

System: You are an expert constructing a taxonomy from a list of concepts. Given a list of concepts,construct a taxonomy by creating a list of their parent-child relationships.


Human: Concepts: Law; Sociology; Economics; Psychology; Law, social and behavioral sciences; Ethnography; Anthropology
Relationships: 
AI: Law is a subtopic of Law, social and behavioral sciences; Ethnography is a subtopic of Anthropology; Psychology is a subtopic of Law, social and behavioral sciences; Sociology is a subtopic of Law, social and behavioral sciences; Anthropology is a subtopic of Law, social and behavioral sciences; Economics is a subtopic of Law, social and behavioral sciences


Human: Concepts: Internet communications tools; Simple Object Access Protocol (SOAP); Service discovery and interfaces; Web crawling; Online advertising; Secure online transactions; Web data description languages; Social advertising; Social tagging; Deep web; Content match advertising; Personalization; Web Ontology Lan

In [87]:
for test_group in tqdm(test_groups):
    concepts, edges = get_nodes_edges(pairs_df, test_group)
    group_examples = generate_train_examples(pairs_df, few_shot_numbers)

    print(llm.get_num_tokens_from_messages(prompt.format_messages(
        concepts=concepts,
        group_examples=group_examples,
    )))
    result = call_chain(chain, concepts, group_examples)
    results.append({
        'group': test_group,
        'nodes': concepts,
        'result': result,
    })

  0%|          | 0/15 [00:00<?, ?it/s]

1845


  7%|▋         | 1/15 [00:24<05:36, 24.04s/it]

[['Probability and statistics is a parent topic for all the concepts'], [' Bayesian computation, Bayesian networks, Gibbs sampling, Markov networks, Markov processes, Markov-chain Monte Carlo convergence measures, Markov-chain Monte Carlo methods, Metropolis-Hastings algorithm are subtopics of Probabilistic reasoning algorithms'], [' Hypothesis testing and confidence interval computation, Maximum likelihood estimation, Regression analysis, Robust regression are subtopics of Statistical paradigms'], [' Resampling methods, Jackknifing, Bootstrapping are subtopics of Nonparametric statistics'], [' Density estimation, Kernel density estimators are subtopics of Nonparametric representations'], [' Multivariate statistics, Cluster analysis, Dimensionality reduction are subtopics of Exploratory data analysis'], [' Factor graphs, Decision diagrams are subtopics of Probabilistic representations'], [' Causal networks, Bayesian networks, Markov networks are subtopics of Equational models'], [' Con

 13%|█▎        | 2/15 [00:26<02:27, 11.32s/it]

...

[['Design for debug', 'Hardware validation'], [' Post-manufacture validation and debug', 'Hardware validation'], [' Coverage metrics', 'Functional verification'], [' Bug fixing (hardware)', 'Bug detection, localization and diagnosis'], [' Assertion checking', 'Functional verification'], [' Bug detection, localization and diagnosis', 'Hardware validation'], [' Power and thermal analysis', 'Physical verification'], [' Timing analysis and sign-off', 'Physical verification'], [' Semi-formal verification', 'Functional verification'], [' Layout-versus-schematics', 'Physical verification'], [' Transaction-level verification', 'Functional verification'], [' Equivalence checking', 'Physical verification'], [' Theorem proving and SAT solving', 'Model checking'], [' Design rule checking', 'Physical verification'], [' Simulation and emulation', 'Hardware validation'], [' Model checking', 'Functional verification'], [' Functional verification', 'Hardware validation'], [' Physical verification', 'Ha

 93%|█████████▎| 14/15 [05:03<00:19, 19.19s/it]

[['Hardware test', 'Online test and diagnostics'], [' Test data compression', 'Online test and diagnostics'], [' Testing with distributed and parallel systems', 'Online test and diagnostics'], [' Test-pattern generation and fault simulation', 'Online test and diagnostics'], [' Fault models and test metrics', 'Online test and diagnostics'], [' Hardware reliability screening', 'Online test and diagnostics'], [' Design for testability', 'Online test and diagnostics'], [' Analog, mixed-signal and radio frequency test', 'Hardware test'], [' Built-in self-test', 'Hardware test'], [' Board- and system-level test', 'Hardware test'], [' Memory test and repair', 'Hardware test'], [' Defect-based test', 'Hardware test\n']]
3130


100%|██████████| 15/15 [05:26<00:00, 21.75s/it]

[['Carbon based electronics', 'Emerging technologies'], [' Circuit substrates', 'Emerging technologies'], [' Quantum dots and cellular automata', 'Emerging technologies'], [' Quantum communication and cryptography', 'Quantum technologies'], [' Quantum error correction and fault tolerance', 'Quantum technologies'], [' Single electron devices', 'Emerging technologies'], [' Flexible and printable circuits', 'Emerging technologies'], [' III-V compounds', 'Emerging technologies'], [' Memory and dense storage', 'Emerging technologies'], [' Quantum computation', 'Quantum technologies'], [' Biology-related information processing', 'Bio-embedded electronics'], [' Microelectromechanical systems', 'Electromechanical systems'], [' Nanoelectromechanical systems', 'Electromechanical systems'], [' Spintronics and magnetic technologies', 'Emerging technologies'], [' Tunneling devices', 'Emerging technologies'], [' Reversible logic', 'Emerging technologies'], [' Emerging architectures', 'Emerging techn

In [88]:
len(results)

15

In [103]:
test_groups

array(['10002950_2', '10002950_3', '10002950_5', '10002951_5',
       '10002978_3', '10002978_9', '10003120_3', '10003456_2',
       '10010147_3', '10010147_5', '10010405_4', '10010583_2',
       '10010583_7', '10010583_8', '10010583_10'], dtype=object)

In [89]:
result_pairs = []
for result in results:
    for edge in result['result']:
        result_pairs.append({
            'group': result['group'],
            'child': edge['child'],
            'parent': edge['parent'],
        })

In [110]:
df = pd.DataFrame(result_pairs)
df.to_csv('./results/gpt-3/ccs/gpt4.csv', index=None)

# Evluation

In [19]:
from utils import (
    convert_to_ancestor_graph, 
    maximum_absorbance, 
    dataframe_to_ancestor_graph, 
    evaluate_groups,
    maximum_likelihood,
    maximum_branching,
    majority_voting
)
import numpy as np

In [3]:
actual_tree='data/bansal_wordnet_true_pairs.csv' # bansal_wordnet_true_pairs acm_ccs_clean
data_dir = f'./results/gpt-3/wordnet/'

## Get individual best result

In [9]:
dfs = []
num_generations = 5

for i in range(1, num_generations + 1):
    filename = f'{data_dir}/results_{i}.csv'
    df = pd.read_csv(filename)
    df['child'] = df['child'].apply(lambda x: x.replace(' ', '_'))
    df['parent'] = df['parent'].apply(lambda x: x.replace(' ', '_'))
    dfs.append(df)

# get ground truth
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['type'] == 'test']
df_actual = dataframe_to_ancestor_graph(df_actual)

100%|██████████| 114/114 [00:00<00:00, 1070.28it/s]


In [11]:
recalls = []
precisions = []
f1s = []

for df in dfs:
    df = dataframe_to_ancestor_graph(df)
    recall, precision, f1 = evaluate_groups(df_actual, df)

    recalls.append(recall)
    precisions.append(precision)
    f1s.append(f1)

100%|██████████| 114/114 [00:00<00:00, 881.62it/s]


In [14]:
print("Best Recall: ", np.max(recalls))
print("Best Precision: ", np.max(precisions))
print("Best F1: ", np.max(f1s))

Best Recall:  0.5364916307655639
Best Precision:  0.646368967793948
Best F1:  0.572489046346162


## Combine Multiple Predictions (Maximum branching)

In [42]:
dfs = []
num_generations = 5

for i in range(1, num_generations + 1):
    filename = f'{data_dir}/results_{i}.csv'
    df = pd.read_csv(filename)
    df['child'] = df['child'].apply(lambda x: x.replace(' ', '_'))
    df['parent'] = df['parent'].apply(lambda x: x.replace(' ', '_'))
    dfs.append(df)

df = pd.concat(dfs)


In [43]:
rows = []
columns = df.columns.tolist()

for group_name, group_df in df.groupby(columns):
    count = group_df.shape[0]
    new_row = {
        columns[0]: group_name[0],
        columns[1]: group_name[1],
        columns[2]: group_name[2],
        'predict': count,
    }
    rows.append(new_row)

merged_df = pd.DataFrame(rows)

In [44]:
def convert_to_ancestor_graph(G):
    '''Converts a (parent) tree to a graph with edges for all ancestor relations in the tree.'''
    G_anc = nx.DiGraph()
    for node in G.nodes():
        for anc in nx.ancestors(G, node):
            G_anc.add_edge(anc, node)
    return G_anc

import networkx as nx

forest=[]
for g in tqdm(list(set(merged_df.group))):
    df = maximum_branching(merged_df, g)
    forest.append(df)
res_v2=pd.concat(forest, ignore_index=True)



100%|██████████| 114/114 [00:00<00:00, 431.74it/s]


In [45]:
df_actual=pd.read_csv(actual_tree)
df_actual=df_actual[df_actual['type'] == 'test']
df_actual['child'] = df_actual['child'].apply(lambda x: x.replace(' ', '_'))
df_actual['parent'] = df_actual['parent'].apply(lambda x: x.replace(' ', '_'))

In [46]:
df_actual = dataframe_to_ancestor_graph(df_actual)

100%|██████████| 114/114 [00:00<00:00, 1101.16it/s]


In [47]:
recall, precision, f1 = evaluate_groups(df_actual, res_v2)
print("Recall: ", recall, "Precision: ", precision, "F1: ", f1)

100%|██████████| 114/114 [00:00<00:00, 822.05it/s]

Recall:  0.6645223913488745 Precision:  0.6276596550517497 F1:  0.630155470051838


In [36]:
df_actual[df_actual.group=='10010147_3']

,parent,child,group,compare


In [26]:
res_v2[res_v2.group=='10010147_3']

,parent,child,group,compare


In [37]:
recall = []
precision = []
f1 = []
for group in tqdm(list(set(df_actual.group))):
    group_actual = df_actual[df_actual.group == group]
    group_pred = res_v2[res_v2.group == group]
    recall.append(len(group_actual.merge(group_pred, on='compare')) / len(group_actual))
    precision.append(len(group_actual.merge(group_pred, on='compare')) / len(group_pred))
    if precision[-1] + recall[-1] == 0:
        f1.append(0)
    else:
        f1.append(2 * (precision[-1] * recall[-1]) / (precision[-1] + recall[-1]))

100%|██████████| 114/114 [00:00<00:00, 494.88it/s]


In [38]:
import numpy as np
np.mean(recall)

0.48965134192568305

In [39]:
np.mean(precision)

0.7506532177798615

In [40]:
np.mean(f1)

0.5704412106495687